In [4]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import networkx as nx
%matplotlib inline


In [5]:
import cvxpy as cp

In [6]:
plt.rcParams['figure.figsize'] = [15, 10]
sns.set_style('whitegrid')
plt.rcParams['font.size'] = 20.0
plt.rcParams['xtick.labelsize'] = 20.0
plt.rcParams['ytick.labelsize'] = 20.0
import pickle

# Generate synthetic data

In [7]:
n = 20
ground_truth_asymm = np.random.uniform(0, 0.5, (n, n))
ground_truth_mat = ground_truth_asymm + ground_truth_asymm.T

In [8]:
def gen_sample_mat(ground_truth_mat):
    '''
    Generate a sample matrix from a ground truth matrix
    inputs:
        ground_truth_mat: A n by n matrix of probabilities
    outputs:    
        A: a n by n matrix whose entries are 1 
        with probability ground_truth_mat[i, j] and 0 otherwise
    '''
    n = ground_truth_mat.shape[0]
    A = np.zeros_like(ground_truth_mat)
    for i in range(n):
        for j in range(i + 1):
            sample = np.random.binomial(1, ground_truth_mat[i, j])
            A[i, j] = sample
            A[j, i] = sample
    return A

In [9]:
sample_mats = [gen_sample_mat(ground_truth_mat) for _ in range(11)]

scale = np.sum(sample_mats[0])
for i in range(len(sample_mats)):
    if i > 0:
        s = np.sum(sample_mats[i])
        rescale = scale / s 
        sample_mats[i] = rescale * sample_mats[i]

In [10]:
for i in range(11): 
    print(np.sum(sample_mats[i]))

221.0
220.99999999999994
221.00000000000003
221.0
221.00000000000006
221.0
221.0
221.0
221.0
221.00000000000006
221.0


Notice that the sum of the entries of each matrix is the same, and is near 0.5 * 400. 

## Define the convex program

For now, we will just set delta manually to be O(sqrt(degree)). Here, the degree is roughly 10, so we set delta to be sqrt(10).

In [18]:
m = len(sample_mats)
n = sample_mats[0].shape[0]
delta = np.sqrt(10.0)

eta = cp.Variable(m, nonneg=True)
# partial_traces = cp.Variable(n)

# define the Y_diff matrix
X_0 = eta[0] * sample_mats[0]
X_arrs = []
for i in range(1, len(sample_mats)): 
    X_arrs.append(eta[i] * (X_0 - sample_mats[i]))
Y = X_0 + np.sum(X_arrs, axis=0)
Y_diff = Y - delta * np.eye(n)

zero_mat = np.zeros_like(Y_diff)
mat_max = cp.atoms.max(Y_diff, zero_mat)
# max_elemwise(Y_diff, zero_mat)
obj = cp.atoms.affine.trace(mat_max)
objective = cp.Minimize(obj)

# obj = cp.atoms.lambda_sum_largest(Y_diff, 1)
# for i in range(2, n + 1):
#     obj2 = cp.atoms.lambda_sum_largest(Y_diff, i)
#     obj = cp.atoms.maximum(obj, obj2)

# partial_traces = [[]] * n
# for j in range(1, n + 1): 
#     partial_trace = cp.atoms.lambda_sum_largest(Y_diff, j)
#     partial_traces[j - 1] = partial_trace

# max_traces = cp.atoms.elementwise.maximum(partial_traces)
# objective = cp.Minimize(obj)



TypeError: only integer scalar arrays can be converted to a scalar index

In [17]:
cp.atoms.elementwise?

Type:        module
String form: <module 'cvxpy.atoms.elementwise' from '/Users/akhiljalan/opt/anaconda3/lib/python3.9/site-packages/cvxpy/atoms/elementwise/__init__.py'>
File:        ~/opt/anaconda3/lib/python3.9/site-packages/cvxpy/atoms/elementwise/__init__.py
Docstring:  
Copyright 2013 Steven Diamond

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.


In [63]:
%%time
prob = cp.Problem(
    objective, 
    [cp.sum(eta) == 1.0]
)

CPU times: user 375 µs, sys: 508 µs, total: 883 µs
Wall time: 604 µs


In [66]:
%%time
prob.solve(verbose=True, solver=cp.SCS)

# eugh, this is not DCP. Why?

                                     CVXPY                                     
                                     v1.2.1                                    
(CVXPY) Mar 01 01:45:34 PM: Your problem has 11 variables, 1 constraints, and 0 parameters.
(CVXPY) Mar 01 01:45:34 PM: It is compliant with the following grammars: 
(CVXPY) Mar 01 01:45:34 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Mar 01 01:45:34 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.


DCPError: Problem does not follow DCP rules. Specifically:
The objective is not DCP. Its following subexpressions are not:
Promote(var565[1], (20, 20)) @ (Promote(var565[0], (20, 20)) @ [[1. 0. 1. 1. 1. 1. 0. 1. 1. 0. 1. 0. 1. 0. 1. 0. 1. 1. 0. 1.]
 [0. 1. 1. 0. 0. 1. 0. 0. 1. 1. 1. 0. 1. 1. 1. 0. 1. 0. 1. 1.]
 [1. 1. 1. 1. 0. 0. 0. 1. 1. 1. 1. 0. 1. 1. 1. 1. 0. 1. 0. 1.]
 [1. 0. 1. 0. 0. 1. 0. 0. 1. 0. 1. 1. 0. 1. 0. 0. 1. 1. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 0. 1. 1. 0. 1. 0. 1. 1. 1.]
 [1. 1. 0. 1. 0. 1. 1. 1. 1. 0. 0. 1. 1. 0. 1. 0. 0. 1. 1. 1.]
 [0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 1. 1. 1. 0. 0. 0. 1. 1. 0. 0.]
 [1. 0. 1. 0. 0. 1. 1. 1. 0. 0. 1. 0. 1. 0. 1. 1. 0. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 0. 0. 1. 1. 1. 0. 1. 0. 1. 1. 1. 1. 1. 1.]
 [0. 1. 1. 0. 1. 0. 0. 0. 1. 0. 1. 0. 1. 1. 0. 1. 0. 0. 1. 0.]
 [1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 1. 1. 0.]
 [0. 0. 0. 1. 0. 1. 1. 0. 0. 0. 0. 1. 1. 1. 0. 1. 1. 1. 0. 0.]
 [1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0.

In [67]:
obj_simple = cp.Minimize(cp.trace(Y_diff))

In [68]:
%%time
prob = cp.Problem(
    obj_simple, 
    [cp.sum(eta) == 1.0]
)

CPU times: user 157 µs, sys: 1 µs, total: 158 µs
Wall time: 163 µs


In [69]:
%%time
prob.solve(verbose=True, solver=cp.SCS)

                                     CVXPY                                     
                                     v1.2.1                                    
(CVXPY) Mar 01 01:46:14 PM: Your problem has 11 variables, 1 constraints, and 0 parameters.
(CVXPY) Mar 01 01:46:14 PM: It is compliant with the following grammars: 
(CVXPY) Mar 01 01:46:14 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Mar 01 01:46:14 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.


DCPError: Problem does not follow DCP rules. Specifically:
The objective is not DCP. Its following subexpressions are not:
Promote(var565[1], (20, 20)) @ (Promote(var565[0], (20, 20)) @ [[1. 0. 1. 1. 1. 1. 0. 1. 1. 0. 1. 0. 1. 0. 1. 0. 1. 1. 0. 1.]
 [0. 1. 1. 0. 0. 1. 0. 0. 1. 1. 1. 0. 1. 1. 1. 0. 1. 0. 1. 1.]
 [1. 1. 1. 1. 0. 0. 0. 1. 1. 1. 1. 0. 1. 1. 1. 1. 0. 1. 0. 1.]
 [1. 0. 1. 0. 0. 1. 0. 0. 1. 0. 1. 1. 0. 1. 0. 0. 1. 1. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 0. 1. 1. 0. 1. 0. 1. 1. 1.]
 [1. 1. 0. 1. 0. 1. 1. 1. 1. 0. 0. 1. 1. 0. 1. 0. 0. 1. 1. 1.]
 [0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 1. 1. 1. 0. 0. 0. 1. 1. 0. 0.]
 [1. 0. 1. 0. 0. 1. 1. 1. 0. 0. 1. 0. 1. 0. 1. 1. 0. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 0. 0. 1. 1. 1. 0. 1. 0. 1. 1. 1. 1. 1. 1.]
 [0. 1. 1. 0. 1. 0. 0. 0. 1. 0. 1. 0. 1. 1. 0. 1. 0. 0. 1. 0.]
 [1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 1. 1. 0.]
 [0. 0. 0. 1. 0. 1. 1. 0. 0. 0. 0. 1. 1. 1. 0. 1. 1. 1. 0. 0.]
 [1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0.